In [1]:
%matplotlib inline

### Autograd: 自动求导机制

PyTorch 所有神经网络的核心是 ``autograd`` 包。先简单介绍一下这个包，然后训练第一个简单的神经网络。

``autograd``包为张量上的所有操作提供了自动求导。
它是一个运行时定义的框架，意味着反向传播根据代码确定如何运行，并且每次迭代可以不同。


### 张量 Tensor

``torch.Tensor``是这个包的核心类。如果设置``.requires_grad`` 为``True``，会追踪所有对于该张量的操作。 
完成计算后调用 ``.backward()``自动计算所有梯度，这个张量的所有梯度自动积累到``.grad``属性。

要阻止张量跟踪历史记录，调用``.detach()``方法将其与计算历史记录分离，禁止跟踪将来的计算记录。

为了防止跟踪历史记录（使用内存），将代码块包装在``with torch.no_grad()：``中。
评估模型特别有用，因为模型可能有`requires_grad=True`的可训练参数，但不需要计算梯度。

自动梯度计算中还有另外一个重要类``Function``.

``Tensor``和``Function``互相连接生成一个非循环图，表示和存储完整的计算历史。
每个张量都有一个``.grad_fn``属性，这个属性引用一个创建了``Tensor``的``Function``
（除非用户手动创建这个张量，即这个张量的``grad_fn``是``None``）。

如果需要计算导数，可以在``Tensor``上调用``.backward()``。 
如果``Tensor``是一个标量（即包含一个元素数据），不需要为``backward()``指定任何参数。
如果有更多元素，需要指定一个``gradient`` 参数匹配张量形状。

其他文章可能会看到将Tensor包裹到Variable中提供自动梯度计算，Variable在0.41版中已经被标注为过期了。
现在可以直接使用Tensor，官方文档在这里：(https://pytorch.org/docs/stable/autograd.html#variable-deprecated) 

In [2]:
import torch

创建一个张量并设置 requires_grad=True 追踪计算历史

In [3]:
x = torch.ones(2, 2, requires_grad=True)
print(x)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)


对张量进行操作:



In [4]:
y = x + 2
print(y)

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)


结果``y``已经被计算出来，``grad_fn``已经被自动生成

In [5]:
print(y.grad_fn)

对y进行一个操作



In [6]:
z = y * y * 3
out = z.mean()
print(z, out)

tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>) tensor(27., grad_fn=<MeanBackward0>)


``.requires_grad_( ... )``可以改变现有张量``requires_grad``属性。
如果没有指定，默认输入flag是 ``False``。

In [8]:
a = torch.randn(2, 2)
a = (a * 3) / (a - 1)
print(a.requires_grad)
print(a.grad_fn)

a.requires_grad_(True)
print(a.requires_grad)

b = (a * a).sum()
print(b.grad_fn)

False
None
True


### 梯度 反向传播
``out``是一个纯量（scalar），``out.backward()`` 等于``out.backward(torch.tensor(1))``

In [9]:
out.backward()

print gradients d(out)/dx




In [10]:
print(x.grad)

tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])


``out``叫做*Tensor* “$o$”.得到 $o = \frac{1}{4}\sum_i z_i$,
$z_i = 3(x_i+2)^2$ 和 $z_i\bigr\rvert_{x_i=1} = 27$.

因此,
$\frac{\partial o}{\partial x_i} = \frac{3}{2}(x_i+2)$, 则
$\frac{\partial o}{\partial x_i}\bigr\rvert_{x_i=1} = \frac{9}{2} = 4.5$.

如果有向量值函数 $\vec{y} = f(\vec{x}))$ ，且 $\vec{y}$ 关于 $\vec{x}$ 的梯度是一个雅可比矩阵(Jacobian matrix)：

$J = \begin{pmatrix} \frac{\partial y_{1}}{\partial x_{1}} & \cdots & \frac{\partial y_{1}}{\partial x_{n}} \\ \vdots & \ddots & \vdots \\ \frac{\partial y_{m}}{\partial x_{1}} & \cdots & \frac{\partial y_{m}}{\partial x_{n}} \end{pmatrix}$

`torch.autograd`用来计算vector-Jacobian product。也就是说，给定任一向量 $v=(v_{1}\;v_{2}\;\cdots\;v_{m})^{T}$ ，计算 $v^{T}\cdot J$。如果 $v$ 恰好是标量函数 $l=g(\vec{y})$ 的梯度，也就是说 $v=(\frac{\partial l}{\partial  y_{1}}\;\cdots\;\frac{\partial l}{\partial  y_{m}})^{T}$，那么根据链式法则，vector-Jacobian product 是 $\vec{x}$ 对 $l$ 的梯度：

$J^{T}\cdot v = \begin{pmatrix} \frac{\partial y_{1}}{\partial x_{1}} & \cdots & \frac{\partial y_{m}}{\partial x_{1}} \\ \vdots & \ddots & \vdots \\ \frac{\partial y_{1}}{\partial x_{n}} & \cdots & \frac{\partial y_{m}}{\partial x_{n}} \end{pmatrix} \begin{pmatrix} \frac{\partial l}{\partial y_{1}}\\ \vdots \\ \frac{\partial l}{\partial y_{m}} \end{pmatrix} = \begin{pmatrix} \frac{\partial l}{\partial x_{1}}\\ \vdots \\ \frac{\partial l}{\partial x_{n}} \end{pmatrix}$

注意，$v^{T}\cdot J$ 给出一个行向量，可以通过 $J^{T}\cdot v$ 将其视为列向量

vector-Jacobian product 使得外部梯度返回到具有非标量输出的模型变得非常方便。

现在来看一个vector-Jacobian product的例子

In [11]:
x = torch.randn(3, requires_grad=True)
y = x * 2
while y.data.norm() < 1000:
    y = y * 2
print(y)

tensor([  812.7551, -1214.2366,  -864.7861], grad_fn=<MulBackward0>)


`y`不再是个标量。
`torch.autograd`无法直接计算完整雅可比行列，如果只想要vector-Jacobian product，只需将向量作为参数传入`backward`：

In [12]:
gradients = torch.tensor([0.1, 1.0, 0.0001], dtype=torch.float)
y.backward(gradients)
print(x.grad)

tensor([5.1200e+01, 5.1200e+02, 5.1200e-02])


如果``.requires_grad=True``，又不希望计算autograd，可以将变量包裹在 ``with torch.no_grad()``中:

In [13]:
print(x.requires_grad)
print((x**2).requires_grad)
with torch.no_grad():
    print((x**2).requires_grad)

True
True
False


**稍后阅读:**

 ``autograd`` 和 ``Function`` 的官方文档 https://pytorch.org/docs/autograd

